## Algoritmo genético que minimiza a função
## $f(x, y) =  20 + x^{2} + y^{2} - 10[cos(2 \pi x)+cos(2 \pi y)]$

* variáveis $x$ e $y$ no intervalo real [ -5.12; 5.12 ]

<!--
#teste forca bruta
from math import cos, pi
min_x = min_y = minimo = 10**10     

for x in np.arange(-5.12, 5.12, 0.01):
    for y in np.arange(-5.12, 5.12, 0.01):
        v = 20 + x**2 + y**2 - 10*(cos(2*pi*x)+cos(2*pi*y))
        if v < minimo:
            minimo = v
            min_x = x
            min_y = y
            
display(minimo)
display(min_x)
display(min_y)

'''
max: 0.0
x: -1.092459456231154e-13
y: -1.092459456231154e-13
'''
-->

In [81]:
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter, LogLocator

import numpy as np
import pandas as pd

import copy
import random
from math import cos, pi
from time import sleep

from ipywidgets import *
import ipywidgets as widgets

from IPython.display import display, HTML

class Individuo():   
    
    def __init__(self, geracao=1, **kwargs):
        if geracao == 1:
            self.cromossomo = np.array([-5.12 + random.random() * (5.12-(-5.12)) for i in range(2)])
        else:
            self.cromossomo = np.empty(2)
        self.geracao = geracao
        self.aptidao = self.x = self.y = 0
        self.limite_sup = 5.12
        self.limite_inf = -5.12
                        
    def __repr__(self): # formata representacao do objeto quanto utiliza print/display
        return "{} - Apt:{:8.3f} - x1:{:8.3f} - x2:{:8.3f} - Geracao: {}".format(self.cromossomo, self.aptidao, 
                                                                  self.x, self.y, self.geracao)
    
    def avaliacao(self):
        x = self.x = self.cromossomo[0]
        y = self.y = self.cromossomo[1]
        self.aptidao = -1*(20 + x**2 + y**2 - 10*(cos(2*pi*x) + cos(2*pi*y))) + 100
        
    def cruzamento(self, outro_individuo, metodo):
        filhos = []
        if metodo == 'radcliffe':
            filhos = [Individuo(geracao=self.geracao+1), Individuo(geracao=self.geracao+1)]
            for i in range(len(self.cromossomo)):
                beta = random.random()
                filhos[0].cromossomo[i] = beta * self.cromossomo[i] + (1-beta) * outro_individuo.cromossomo[i] 
                filhos[1].cromossomo[i] = (1-beta) * self.cromossomo[i] +  beta * outro_individuo.cromossomo[i] 
                
        else:
            potenciais_filhos = [Individuo(geracao=self.geracao+1), Individuo(geracao=self.geracao+1), 
                                  Individuo(geracao=self.geracao+1)]
            for i in range(len(self.cromossomo)):
                potenciais_filhos[0].cromossomo[i] = 0.5 * self.cromossomo[i] + 0.5 * outro_individuo.cromossomo[i]
                if potenciais_filhos[0].cromossomo[i] < self.limite_inf or potenciais_filhos[0].cromossomo[i] > self.limite_sup:
                    potenciais_filhos[0].cromossomo[i] = -5.12 + random.random() * (5.12-(-5.12))
                    
                potenciais_filhos[1].cromossomo[i] = 1.5 * self.cromossomo[i] - 0.5 * outro_individuo.cromossomo[i]
                if potenciais_filhos[1].cromossomo[i] < self.limite_inf or potenciais_filhos[1].cromossomo[i] > self.limite_sup:
                    potenciais_filhos[1].cromossomo[i] = -5.12 + random.random() * (5.12-(-5.12))
                                       
                potenciais_filhos[2].cromossomo[i] = -0.5 * self.cromossomo[i] + 1.5 * outro_individuo.cromossomo[i]
                if potenciais_filhos[2].cromossomo[i] < self.limite_inf or potenciais_filhos[2].cromossomo[i] > self.limite_sup:
                    potenciais_filhos[2].cromossomo[i] = -5.12 + random.random() * (5.12-(-5.12))
                                       
            for i in range(len(potenciais_filhos)):
                potenciais_filhos[i].avaliacao()
            
            potenciais_filhos = sorted(potenciais_filhos,
                               key =  lambda individuo: individuo.aptidao,
                               reverse = True)
            potenciais_filhos = np.array(potenciais_filhos)
            filhos = potenciais_filhos[0:2]
        
        return filhos
    
    def mutacao(self, taxa_mutacao):        
        if random.random() < taxa_mutacao:
            self.cromossomo[random.randint(0, 1)] = -5.12 + random.random() * (5.12-(-5.12))
            
        return self
    

class AlgoritmoGenetico():
    
    def __init__(self, tam_populacao, taxa_cruzamento, taxa_mutacao,
                qtd_geracoes, selecao, elitismo, tam_torneio=0, **kwargs):
        self.tam_populacao = tam_populacao
        self.populacao = np.array([])
        self.taxa_cruzamento = taxa_cruzamento
        self.taxa_mutacao = taxa_mutacao
        self.qtd_geracoes = qtd_geracoes
        self.geracao = 0
        self.selecao = selecao
        self.tam_torneio = tam_torneio
        self.elitismo = elitismo
        self.lista_solucao = []
        self.melhor_solucao = 0
        self.cruzamento = cruzamento
        self.metodo_cruzamento = metodo_cruzamento
    
    def inicializa_populacao(self):   
        lista = []
        for i in range(self.tam_populacao):
            lista.append(Individuo())
        self.populacao = np.array(lista)
        self.melhor_solucao = self.populacao[0]    
        
    def ordena_populacao(self):
        self.populacao = sorted(self.populacao,
                               key =  lambda individuo: individuo.aptidao,
                               reverse = True)
        self.populacao = np.array(self.populacao)
        
    def calcula_aptidao(self):
        for ind in self.populacao:
            ind.avaliacao()
    
    def melhor_individuo(self, individuo):
        if individuo.aptidao < self.melhor_solucao.aptidao:
            self.melhor_solucao = individuo
    
    def soma_aptidoes(self):
        soma = 0
        for i in self.populacao:
            soma += i.aptidao
        return soma
            
    def seleciona_pais(self, soma_aptidoes=0):
        '''
        retorna um array com o par de pais com base no tipo de selecao
        '''
        if self.selecao == 'roleta':
            r_pais = [random.random() * soma_aptidoes]
            while True:
                r = random.random() * soma_aptidoes
                if r not in r_pais:
                    r_pais.append(r)
                    break
                    
            pais = []
            for r in r_pais:
                pai = -1
                i = 0
                soma = 0
                while i < len(self.populacao) and soma < r:
                    soma += self.populacao[i].aptidao
                    pai += 1
                    i += 1
                pais.append(self.populacao[pai])
            return pais
        
        elif self.selecao == 'torneio':
            indices_torneio = random.sample(range(0, self.tam_populacao), self.tam_torneio)
            torneio = self.populacao[indices_torneio]
            torneio = sorted(torneio, key=lambda i: i.aptidao, reverse=True)
            return [torneio[0], torneio[1]]
        
    
    def seleciona_elitismo(self):
        '''
        retorna lista contendo os individuos elitistas
        '''
        self.ordena_populacao()
        return self.populacao[0:self.elitismo]
    
    

########## entradas dos parametros ##########

print('\n\nTamanho da população:')
def f(populacao=50):
    return populacao
populacao = interactive(f, populacao=(10, 100, 10))
display(populacao)

print('\n\nProbabilidade de cruzamento:')
def f(cruzamento=95.0):
    return cruzamento
cruzamento = interactive(f, cruzamento=(5.0, 95.0, 0.5))
display(cruzamento)

print('\n\nProbabilidade de mutação:')
def f(mutacao=0.1):
    return mutacao
mutacao = interactive(f, mutacao=(1.0, 10.0, 0.5))
display(mutacao)

print('\n\nQuantidade de gerações:')
def f(geracoes=30):
    return geracoes
geracoes = interactive(f, geracoes=(10, 100))
display(geracoes)

print('\n\nMétodo de seleção:')
def f(selecao):
    return selecao
selecao = interactive(f, selecao=['Torneio', 'Roleta'])
display(selecao)

print("""\n\nTamanho do Torneio:\n *ignorar caso o 'Método de seleção' selecionado seja diferente de 'Torneio'*""")
def f(tam_torneio=10):
    return tam_torneio
tam_torneio = interactive(f, tam_torneio=(2, 50, 2))
display(tam_torneio)

print('\n\nTamanho do elitismo:')
def f(elitismo=1):
    return elitismo
elitismo = interactive(f, elitismo=(0, 10))
display(elitismo)

print('\n\nMétodo de cruzamento:')
def f(metodo_cruzamento):
    return metodo_cruzamento
metodo_cruzamento = interactive(f, metodo_cruzamento=['Radcliffe', 'Wright'])
display(metodo_cruzamento)


########## processamento ##########
def teste():
    ind = Individuo()
    ind.avaliacao()
    
    ind2 = Individuo()
    ind2.avaliacao()
    
    filhos = ind.cruzamento(outro_individuo=ind2, metodo='wright')
    display(ind)
    display(ind2)
    display(filhos)
    return


def processar():
    
    alg = AlgoritmoGenetico(
        tam_populacao = populacao.result,
        taxa_cruzamento = cruzamento.result/100.0,
        taxa_mutacao = mutacao.result/100.0,
        qtd_geracoes = geracoes.result,
        selecao = selecao.result.lower(),
        elitismo = elitismo.result,
        tam_torneio = tam_torneio.result,
        metodo_cruzamento = metodo_cruzamento.result.lower())
    
    ## inicio ##
    
    alg.inicializa_populacao()
    
    melhores_apt = []
    melhores_x = []
    melhores_y = []
    
    for g in range(0, alg.qtd_geracoes):
        #print("Geracao {}".format(g+1))
        
        alg.calcula_aptidao()
        alg.ordena_populacao()
        
        
        nova_popu = []
        soma_apt = alg.soma_aptidoes() if alg.selecao == 'roleta' else 0
        for c in range(0, alg.tam_populacao, 2):
            pais = alg.seleciona_pais(soma_apt)
            if random.random() <= alg.taxa_cruzamento:
                filhos = pais[0].cruzamento(pais[1], alg.metodo_cruzamento)
                for filho in filhos:
                    nova_popu.append(filho.mutacao(alg.taxa_mutacao))
            else:
                for pai in pais:
                    nova_popu.append(pai)
        
        elitistas = alg.seleciona_elitismo()
        indices = random.sample(range(0, alg.tam_populacao), len(elitistas))
        for idx in range(0, len(elitistas)):
            alg.populacao[indices[idx]] = elitistas[idx]
            
        alg.populacao = np.array(nova_popu)
        
        alg.calcula_aptidao()
        alg.ordena_populacao()
        
        melhor = copy.deepcopy(alg.populacao[0])
        melhor.aptidao = -1 * (melhor.aptidao - 100)
        alg.lista_solucao.append(melhor)
        alg.melhor_individuo(melhor)
        
        melhores_apt.append(melhor.aptidao)
        melhores_x.append(melhor.x)
        melhores_y.append(melhor.y)
        
        
    string = "<h4><strong>Geração da melhor solução:</strong> {:}</h4>".format(alg.melhor_solucao.geracao)
    display(HTML(string))
    
    string = "<h4><strong>Melhor solução:</strong> f({:2.2f}, {:2.2f}) = {:3.3f}</h4>" \
                .format(alg.melhor_solucao.x, alg.melhor_solucao.y, alg.melhor_solucao.aptidao)
    display(HTML(string))
    ########## graficos ##########
    
    l = plt.plot(list(range(1, alg.qtd_geracoes+1)), melhores_apt, 'bo')
    plt.setp(l, markersize=5, linestyle="-", label='Teste')
    plt.title('Geração x Aptidão')
    plt.xlabel('Geração')
    plt.ylabel('Aptidão')
    plt.show()
    
    
    x = np.arange(-5.12, 5.12, 0.01) #### quanto menor o intervalo,mais preciso o grafico mas mais lento fica o processamento
    y = np.arange(-5.12, 5.12, 0.01)
    X, Y = np.meshgrid(x, y)
    Z = 20.0 + X**2 + Y**2 - 10 * (np.cos(2*pi*X) + np.cos(2*pi*Y))
    
    
    fig = plt.figure(figsize=plt.figaspect(0.3), dpi=100)

    # grafico da superficie
    ax = fig.add_subplot(1, 2, 1, projection='3d')
    surf = ax.plot_surface(X, Y, Z, cmap='Blues', alpha=0.4)
    ax.zaxis.set_major_locator(LinearLocator(10))
    ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))
    fig.colorbar(surf, shrink=0.5, aspect=5)
    
    ax.scatter(melhores_x, melhores_y, s=10, c='red', label="Mínimos")
    ax.legend()
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Superfície')

    # grafico do contorno
    ax = fig.add_subplot(1, 2, 2)
    contorno = ax.contourf(X, Y, Z, locator=LogLocator(), cmap='Blues')

    ax.scatter(melhores_x, melhores_y, s=10, c='red', label='Mínimos')
    ax.scatter([alg.melhor_solucao.x], [alg.melhor_solucao.y], s=20,
               c='yellow', marker="*", label='Melhor mínimo')
    ax.legend()
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Contorno')
    plt.show()
    
    
    ########## /graficos ##########
    '''
    string = "<h4><strong>Melhor solução:</strong> {:8.3f}</h4>" \
            "<h4><strong>X1:</strong> {:8.3f}</h4>" \
            "<h4><strong>X2:</strong> {:8.3f}</h4>", color \
            "<h4><strong>Geração:</strong> {}</h4>".format(
        alg.melhor_solucao.aptidao, alg.melhor_solucao.x1, alg.melhor_solucao.x2,
        alg.melhor_solucao.geracao)
    display(HTML(string))
    
    '''
        
    
    
widgets.interact_manual.opts['manual_name'] = 'Processar algoritmo' # muda texto do botao
interact_manual(processar); # metodo a executar quando pressionar o botao



Tamanho da população:


interactive(children=(IntSlider(value=50, description='populacao', min=10, step=10), Output()), _dom_classes=(…



Probabilidade de cruzamento:


interactive(children=(FloatSlider(value=95.0, description='cruzamento', max=95.0, min=5.0, step=0.5), Output()…



Probabilidade de mutação:


...



Tamanho do elitismo:


interactive(children=(IntSlider(value=1, description='elitismo', max=10), Output()), _dom_classes=('widget-int…



Método de cruzamento:


interactive(children=(Dropdown(description='metodo_cruzamento', options=('Radcliffe', 'Wright'), value='Radcli…

interactive(children=(Button(description='Processar algoritmo', style=ButtonStyle()), Output()), _dom_classes=…